## Data Science Level 2 Practical Problem 2305-3

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_raw02 = pd.read_csv('lv2-2305-3.csv')
print(df_raw02.shape)
df_raw02.info()

(32686, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32686 entries, 0 to 32685
Data columns (total 9 columns):
OBS_DT            32686 non-null object
RADIATION         32686 non-null float64
TEMPERATURE       32686 non-null float64
PRESSURE          32686 non-null float64
HUMIDITY          32686 non-null float64
WIND_DIRECTION    32686 non-null float64
WIND_SPEED        32686 non-null float64
TIME_SUNRISE      32686 non-null object
TIME_SUNSET       32686 non-null object
dtypes: float64(6), object(3)
memory usage: 2.2+ MB


### Q1

In [3]:
df240 = df_raw02.copy()
df240['TIME'] = df240['OBS_DT'].map(lambda x: x[-8:])
df241 = df240[(df240['TIME']>=df240['TIME_SUNRISE']) & \
      (df240['TIME']<=df240['TIME_SUNSET'])].copy()

In [4]:
df241.shape

(15608, 10)

In [5]:
df241.corr(method='pearson')

,RADIATION,TEMPERATURE,PRESSURE,HUMIDITY,WIND_DIRECTION,WIND_SPEED
RADIATION,1.000000,0.645984,0.234832,-0.393566,-0.154165,0.125902
TEMPERATURE,0.645984,1.000000,0.362515,-0.545887,-0.195206,0.010866
PRESSURE,0.234832,0.362515,1.000000,-0.283555,-0.250171,-0.162957
HUMIDITY,-0.393566,-0.545887,-0.283555,1.000000,0.047233,-0.099915
WIND_DIRECTION,-0.154165,-0.195206,-0.250171,0.047233,1.000000,0.076879
WIND_SPEED,0.125902,0.010866,-0.162957,-0.099915,0.076879,1.000000


In [6]:
list_tmp = []
list_tmp.append(df241.corr(method='pearson').loc['PRESSURE','RADIATION'])
list_tmp.append(df241.corr(method='pearson').loc['RADIATION','WIND_SPEED'])
list_tmp.append(df241.corr(method='pearson').loc['HUMIDITY','RADIATION'])

ans24 = np.max(np.abs(list_tmp))
ans24

0.39356555779946645

In [8]:
print("ANS : ", np.round(ans24, 3))

ANS :  0.394


### Q2

In [9]:
df250 = df_raw02.copy()
df250['DATE'] = df250['OBS_DT'].map(lambda x: x[:10])

In [10]:
df251 = df250.groupby(['DATE'])['TEMPERATURE'].max().reset_index()

In [11]:
df251.shape

(118, 2)

In [12]:
df252 = pd.concat([df251, pd.DataFrame({'DATE':['2022-09-30','2022-11-30',\
                                                '2022-12-06','2022-12-07'],
             'TEMPERATURE':[None,None,None,None]})], axis=0, ignore_index=True)
df252 = df252.sort_values('DATE')
df252.index = range(df252.shape[0])

In [13]:
list_date = ['2022-09-30', '2022-11-30', '2022-12-06']
idx_tmp00 = df252.loc[df252['DATE'].isin(list_date),:].index
for i in idx_tmp00:
    df252.loc[i,'TEMPERATURE'] = df252.loc[i-1, 'TEMPERATURE']*0.7 + \
    df252.loc[i-2, 'TEMPERATURE']*0.3
    print(df252.loc[i,'TEMPERATURE'])

64.0
54.3
45.0


In [14]:
idx_ = df252.loc[df252['DATE']=='2022-12-07',:].index
df252.loc[idx_,'TEMPERATURE'] = \
df252.loc[idx_-2, 'TEMPERATURE'].values[0]*0.7 + \
df252.loc[idx_-3, 'TEMPERATURE'].values[0]*0.3
print(df252.loc[idx_,'TEMPERATURE'])

97    45.0
Name: TEMPERATURE, dtype: float64


In [15]:
ans25 = df252[df252['DATE'].isin(['2022-09-30', '2022-11-30', '2022-12-06','2022-12-07'])]['TEMPERATURE'].mean()
ans25

52.075

In [16]:
print("ANS : ",np.round(ans25, 1))

ANS :  52.1


### Q3

In [17]:
df260 = df_raw02.copy()
df260['DATE'] = df260['OBS_DT'].map(lambda x: x[:10])

In [18]:
df260['H1'] = df260['TIME_SUNRISE'].map(lambda x: int(x[:2]))
df260['m1'] = df260['TIME_SUNRISE'].map(lambda x: int(x[3:5]))
df260['s1'] = df260['TIME_SUNRISE'].map(lambda x: int(x[6:8]))
df260['H2'] = df260['TIME_SUNSET'].map(lambda x: int(x[:2]))
df260['m2'] = df260['TIME_SUNSET'].map(lambda x: int(x[3:5]))
df260['s2'] = df260['TIME_SUNSET'].map(lambda x: int(x[6:8]))

In [19]:
df260['SUN_DURATION'] = 3600*(df260['H2']-df260['H1']) +\
60*(df260['m2']-df260['m1']) + df260['s2']-df260['s1']

In [20]:
df261 = df260.groupby(['DATE','SUN_DURATION'])\
['TEMPERATURE','PRESSURE','HUMIDITY',\
 'WIND_DIRECTION','WIND_SPEED','RADIATION'].mean().reset_index()

In [21]:
df261['day'] = df261['DATE'].map(lambda x: x[-2:])

In [22]:
df_train = df261[(df261['day']!='01') & (df261['day']!='15')].copy()
df_test = df261[(df261['day']=='01') | (df261['day']=='15')].copy()
print(df_train.shape, df_test.shape)

(110, 9) (8, 9)


In [23]:
list_cols = ['SUN_DURATION','TEMPERATURE','PRESSURE',\
             'HUMIDITY','WIND_DIRECTION','WIND_SPEED']

In [24]:
from sklearn.linear_model import LinearRegression
LR00 = LinearRegression()
LR00.fit(df_train.loc[:,list_cols],df_train['RADIATION'])
arr_pred = LR00.predict(df_test.loc[:,list_cols])
arr_act = np.array(df_test['RADIATION'])

In [25]:
ans26 = ((arr_act - arr_pred)**2).mean()
ans26

2364.5251270522867

In [26]:
print("ANS : ", np.round(ans26, 2))

ANS :  2364.53
